In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
import pandas as pd

In [3]:
os.getcwd()

'E:\\PycharmProjects\\Practice Python\\KN\\Tensorflow'

In [4]:
os.listdir()

['.ipynb_checkpoints',
 '1. Tensor Basics.ipynb',
 '2. Neural Networks .ipynb',
 '3. Convolutional Neural Networks with Sequential and Functional API.ipynb',
 '4. Adding Regularization and Dropout.ipynb',
 '5_RNNs,_GRUs,_LSTMS_and_Biidirectionality.ipynb',
 'In Depth Example on Functional API.ipynb',
 'Multi Mnist',
 'Untitled.ipynb']

In [5]:
os.listdir(os.getcwd()+'\Multi Mnist')

['archive.zip',
 'kaggle.json',
 'test.csv',
 'test_images',
 'train.csv',
 'train_images']

In [6]:
os.path.join(os.getcwd(), 'Multi Mnist\train.csv')

'E:\\PycharmProjects\\Practice Python\\KN\\Tensorflow\\Multi Mnist\train.csv'

In [13]:
train_df = pd.read_csv(os.path.join(os.getcwd(), 'Multi Mnist\\train.csv'))
test_df = pd.read_csv(os.path.join(os.getcwd(), 'Multi Mnist\\test.csv'))
train_images = os.getcwd() + "\\Multi Mnist\\train_images\\" + train_df.iloc[:, 0].values
test_images = os.getcwd() + "\\Multi Mnist\\test_images\\" + test_df.iloc[:, 0].values
train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values

In [14]:
def read_image(image_path, label):
    print(image_path)
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    # In older versions you need to set shape in order to avoid error
    # on newer (2.3.0+) the following 3 lines can safely be removed
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels

In [15]:
# HYPERPARAMETERS
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

Tensor("args_0:0", shape=(), dtype=string)
Tensor("args_0:0", shape=(), dtype=string)


In [ ]:
###### Here we cannot use the sequential model becasue sequential model can only map only one input to one output
# But here we have two output.

inputs = keras.Input(shape=(64,64,1))
x = keras.layers.Conv2D(
    filters = 32,
    kernel_size=3,
    padding='same',
    kernel_regularizer = keras.regularizers.l2(WEIGHT_DECAY),
)(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)

x = keras.layers.Conv2D(
    64, 3, kernel_regularizer = keras.regularizers.l2(WEIGHT_DECAY),
)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D()(x)

x = keras.layers.Conv2D(
    64, 3, activation='relu', kernel_regularizer = keras.regularizers.l2(WEIGHT_DECAY),
)(x)

x = keras.layers.Conv2D(
    128, 3, activation='relu')(x)
x = keras.layers.MaxPool2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(64, activation='relu')(x)

output1 = keras.layers.Dense(10, activation='softmax',name='first_num')(x)
output2 = keras.layers.Dense(10,  activation='softmax',name='second_num')(x)

model = keras.Model(inputs=inputs, outputs = [output1, output2])

model.compile(
    optimizer = keras.optimizers.Adam(LEARNING_RATE),
    loss = [
        keras.losses.SparseCategoricalCrossentropy(), 
        keras.losses.SparseCategoricalCrossentropy(),
    ],
    metrics=['accuracy'],
)

model.fit(train_dataset, epochs = 5)
model.evaluate(test_dataset)